In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()



Successfully saved authorization token.


## parameters

In [2]:
# CASE 1: Pakistan Floods 2022

city = 'Shikarpur'
city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
    .filter(ee.Filter.eq('shapeName', city))


date = ee.Date('2022-08-31')



In [11]:


city = 'Ernakulam'
city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
    .filter(ee.Filter.eq('shapeName', city))


date = ee.Date('2022-08-16')


In [15]:




city = 'Nhamatanda'
city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
    .filter(ee.Filter.eq('shapeName', city))


date = ee.Date('2019-03-25')

In [18]:


city = 'Sylhet'
city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
    .filter(ee.Filter.eq('shapeName', city))


date = ee.Date('2022-07-15')

In [19]:

def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])


aoi = city_shp.geometry()

# Load the Sentinel-2 Image Collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                 .filterBounds(aoi) \
                 .filterDate(date, date.advance(1, 'day'), ) \
                 .map(maskS2clouds)
print(sentinel2.size().getInfo())
image = sentinel2.mosaic().clip(aoi)

Map = geemap.Map()
Map.centerObject(city_shp, 10)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'RGB')
Map.addLayer(image, {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 0.3}, 'False Color')
Map.addLayer(city_shp, {}, city)
Map


2


Map(center=[24.918938363868534, 91.98977416508959], controls=(WidgetControl(options=['position', 'transparent_…

In [20]:

# Export function
task = ee.batch.Export.image.toDrive(
        image=image.select(['B2', 'B3', 'B4', 'B8']),
        description=f'{city}_s2_{date.format("YYYY-MM-dd").getInfo()}',
        folder='FMSE',
        region=aoi,
        scale=10,
        maxPixels=1e13
    )
task.start()
print(task.status())

{'state': 'READY', 'description': 'Sylhet_s2_2022-07-15', 'creation_timestamp_ms': 1718269971700, 'update_timestamp_ms': 1718269971700, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'LZ4OVB5WSSCOGSZVANKYQNNZ', 'name': 'projects/earthengine-legacy/operations/LZ4OVB5WSSCOGSZVANKYQNNZ'}
